In [1]:
# Dependencies
import pandas as pd
import numpy as np
import sqlite3
from pyspark import SparkContext
sc =SparkContext()
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
import collections
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Dependencies
import re, string

import nltk
nltk.download("punkt")
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/janelchadiarova/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# Create connections to database
conn = sqlite3.connect("Data/Hotels.db")

#Load the database table into a pandas dataframe
ratings = pd.read_sql_query("select * from ratings;", conn)
conn.close()

# Preview the dataframe
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,Our experience at Rancho Valencia was absolute...,Best romantic vacation ever!!!!,None,None
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,Amazing place. Everyone was extremely warm and...,Sweet sweet serenity,None,None
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,We booked a 3 night stay at Rancho Valencia to...,Amazing Property and Experience,None,None
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,Currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,I live in Md and the Aloft is my Home away fro...,ALWAYS GREAT STAY...,Laurel,MD


In [4]:
#Grouping ratings to get 5 unique ratings
rating_list = ratings['reviews_rating'].tolist()
new_list = []
for rating in rating_list:
    if rating >= 5.0: 
        new_list.append(5.0)
    elif rating >= 4 and rating < 5:
        new_list.append(4.0)
    elif rating >= 3 and rating < 4:
        new_list.append(3.0)
    elif rating >= 2 and rating < 3:
        new_list.append(2.0)
    else:
        new_list.append(1.0)        

In [5]:
# Put all letters in lower case
# Split hotel reviews_rating to "good"/"bad"
ratings["reviews_text"] = ratings["reviews_text"].str.lower()
#ratings["rating"] = np.where(ratings["reviews_rating"]>= 4, 'good', 'bad')
ratings["rating"] = new_list
ratings['reviews_text'] = ratings['reviews_text'].astype(str)
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,rating
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,our experience at rancho valencia was absolute...,Best romantic vacation ever!!!!,None,None,5.0
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,amazing place. everyone was extremely warm and...,Sweet sweet serenity,None,None,5.0
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,we booked a 3 night stay at rancho valencia to...,Amazing Property and Experience,None,None,5.0
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,2.0
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,i live in md and the aloft is my home away fro...,ALWAYS GREAT STAY...,Laurel,MD,5.0


In [6]:
# Remove stop words from the list
stops = stopwords.words("english")
exclude = set(string.punctuation)

In [7]:
words_list = []
preprocessed_text = []
for review in ratings["reviews_text"]:
    
    # Create a list of words per rating after the words are converted to lowercase    
    words = word_tokenize(review)
    
    # Filter to remove stop words and punctuations    
    words2 = [word for word in words if word not in stops and word not in exclude]
    
    # Add the filtered list of words
    words_list.append(words2)
    
    # Convert the list of strings back to one string
    words3 = " ".join(words2)
    
    # Add the filtered list of words
    preprocessed_text.append(words3)
   
ratings["filteredReview"] = preprocessed_text
ratings.head()

,index,name,reviews_date,reviews_rating,reviews_sourceURLs,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,rating,filteredReview
0,0,Rancho Valencia Resort Spa,2013-11-14T00:00:00Z,5.0,www.hotels.com,our experience at rancho valencia was absolute...,Best romantic vacation ever!!!!,None,None,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,2014-07-06T00:00:00Z,5.0,www.hotels.com,amazing place. everyone was extremely warm and...,Sweet sweet serenity,None,None,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,2015-01-02T00:00:00Z,5.0,www.hotels.com,we booked a 3 night stay at rancho valencia to...,Amazing Property and Experience,None,None,5.0,booked 3 night stay rancho valencia play tenni...
3,3,Aloft Arundel Mills,2016-05-15T00:00:00Z,2.0,www.tripadvisor.com,currently in bed writing this for the past hr ...,"Never again...beware, if you want sleep.",Richmond,VA,2.0,currently bed writing past hr 1/2 dogs barking...
4,4,Aloft Arundel Mills,2016-07-09T00:00:00Z,5.0,www.tripadvisor.com,i live in md and the aloft is my home away fro...,ALWAYS GREAT STAY...,Laurel,MD,5.0,live md aloft home away home ... stayed 1 nigh...


In [8]:
# Remove columns that will not be used in the analysis
df = ratings.drop(columns=["reviews_text","reviews_date","reviews_sourceURLs","reviews_title","reviews_userCity","reviews_userProvince"],axis=1)
df.head()

,index,name,reviews_rating,rating,filteredReview
0,0,Rancho Valencia Resort Spa,5.0,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,5.0,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,5.0,5.0,booked 3 night stay rancho valencia play tenni...
3,3,Aloft Arundel Mills,2.0,2.0,currently bed writing past hr 1/2 dogs barking...
4,4,Aloft Arundel Mills,5.0,5.0,live md aloft home away home ... stayed 1 nigh...


In [9]:
#Convert Pandas DataFrame to Spark DataFrame
spark_ratings = sqlContext.createDataFrame(df)
spark_ratings.show(5)

+-----+--------------------+--------------+------+--------------------+
|index|                name|reviews_rating|rating|      filteredReview|
+-----+--------------------+--------------+------+--------------------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|
|    2|Rancho Valencia R...|           5.0|   5.0|booked 3 night st...|
|    3| Aloft Arundel Mills|           2.0|   2.0|currently bed wri...|
|    4| Aloft Arundel Mills|           5.0|   5.0|live md aloft hom...|
+-----+--------------------+--------------+------+--------------------+
only showing top 5 rows



In [10]:
# Create a length column to be used as a future feature 
from pyspark.sql.functions import length
data = spark_ratings.withColumn('length', length(spark_ratings['filteredReview']))
data.show(5)

+-----+--------------------+--------------+------+--------------------+------+
|index|                name|reviews_rating|rating|      filteredReview|length|
+-----+--------------------+--------------+------+--------------------+------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|   112|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|   202|
|    2|Rancho Valencia R...|           5.0|   5.0|booked 3 night st...|   335|
|    3| Aloft Arundel Mills|           2.0|   2.0|currently bed wri...|   125|
|    4| Aloft Arundel Mills|           5.0|   5.0|live md aloft hom...|   186|
+-----+--------------------+--------------+------+--------------------+------+
only showing top 5 rows



## Feature Transformations

In [11]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

In [12]:
#tokenizer = Tokenizer(inputCol="filteredReview", outputCol="token_text")
#tokenized = tokenizer.transform(spark_ratings)
#tokenized.show(3, truncate=False)

In [13]:
#stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
#filtered = stopremove.transform(tokenized)
#filtered.show(3, truncate=False)

In [14]:
#hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token',numFeatures=pow(2,4))
#hashed_df = hashingTF.transform(filtered)
#hashed_df.show(3, truncate=False)

In [15]:
#idf = IDF(inputCol="hash_token", outputCol="idf_token")
#idfModel = idf.fit(hashed_df)
#rescaledData = idfModel.transform(hashed_df)

In [16]:
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='rating',outputCol='label')
tokenizer = Tokenizer(inputCol="filteredReview", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="stop_tokens", outputCol='hash_token',numFeatures=pow(2,4))
idf = IDF(inputCol="hash_token", outputCol="idf_token")

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['label','idf_token', 'length'], outputCol='features')

In [18]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [19]:
 # Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(data)
cleaned = cleaner.transform(data)
cleaned.show(3)

+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|index|                name|reviews_rating|rating|      filteredReview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|
+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|    0|Rancho Valencia R...|           5.0|   5.0|experience rancho...|   112|  0.0|[experience, ranc...|[experience, ranc...|(16,[1,4,5,6,8,9,...|(16,[1,4,5,6,8,9,...|[0.0,0.0,0.184744...|
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|   202|  0.0|[amazing, place, ...|[amazing, place, ...|(16,[2,3,4,5,6,7,...|(16,[2,3,4,5,6,7,...|[0.0,0.0,0.0,0.28...|
|    2|Rancho Valencia R...|           5.0|   5.0|

In [20]:
# Show label of ham spame and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[0.0,0.0,0.184744...|
|  0.0|[0.0,0.0,0.0,0.28...|
|  0.0|[0.0,0.5991695591...|
|  3.0|[3.0,0.2995847795...|
|  0.0|[0.0,1.4979238978...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  0.0|(18,[2,3,4,5,10,1...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,1.4979238978...|
|  0.0|[0.0,0.2995847795...|
|  0.0|[0.0,0.8987543387...|
|  1.0|[1.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  2.0|[2.0,0.5991695591...|
|  0.0|[0.0,0.5991695591...|
|  1.0|[1.0,0.0,0.738976...|
+-----+--------------------+
only showing top 20 rows



In [21]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

In [22]:
# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [23]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show(3)

+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|index|                name|reviews_rating|rating|      filteredReview|length|label|          token_text|         stop_tokens|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------+------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    1|Rancho Valencia R...|           5.0|   5.0|amazing place eve...|   202|  0.0|[amazing, place, ...|[amazing, place, ...|(16,[2,3,4,5,6,7,...|(16,[2,3,4,5,6,7,...|[0.0,0.0,0.0,0.28...|[-63.540398288136...|[0.79744540188972...|       0.0|
| 1000|Hotel Zetta San F...|

In [24]:
# rating: 5 - 0.0, 4 - 1.0, 3 - 2.0, 2 - 3.0, 1 - 4.0
test_results.select(['label', 'prediction']).show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
|  1.0|       1.0|
|  3.0|       2.0|
|  0.0|       0.0|
|  1.0|       1.0|
|  3.0|       4.0|
|  1.0|       1.0|
|  3.0|       2.0|
|  0.0|       0.0|
+-----+----------+
only showing top 10 rows



In [25]:
 # Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting ratings was: ", acc)

Accuracy of model at predicting ratings was:  0.7657299245230653


## Popular Words

In [26]:
df.head()

,index,name,reviews_rating,rating,filteredReview
0,0,Rancho Valencia Resort Spa,5.0,5.0,experience rancho valencia absolutely perfect ...
1,1,Rancho Valencia Resort Spa,5.0,5.0,amazing place everyone extremely warm welcomin...
2,2,Rancho Valencia Resort Spa,5.0,5.0,booked 3 night stay rancho valencia play tenni...
3,3,Aloft Arundel Mills,2.0,2.0,currently bed writing past hr 1/2 dogs barking...
4,4,Aloft Arundel Mills,5.0,5.0,live md aloft home away home ... stayed 1 nigh...


In [27]:
#Filtering Rating 5.0
filtered_5 = df[df['rating'] == 5.0]
list_filtered_5 = filtered_5["filteredReview"].tolist()
list_filtered_5_new = ''.join(list_filtered_5)
#Filtering Rating 4.0
filtered_4 = df[df['rating'] == 4.0]
list_filtered_4 = filtered_4["filteredReview"].tolist()
list_filtered_4_new = ''.join(list_filtered_4)
#Filtering Rating 3.0
filtered_3 = df[df['rating'] == 3.0]
list_filtered_3 = filtered_3["filteredReview"].tolist()
list_filtered_3_new = ''.join(list_filtered_3)
#Filtering Rating 2.0
filtered_2 = df[df['rating'] == 2.0]
list_filtered_2 = filtered_2["filteredReview"].tolist()
list_filtered_2_new = ''.join(list_filtered_2)
#Filtering Rating 1.0
filtered_1 = df[df['rating'] == 1.0]
list_filtered_1 = filtered_1["filteredReview"].tolist()
list_filtered_1_new = ''.join(list_filtered_1)


In [28]:
#Extending stop words list to avoid useless words 
stops_hotel = stopwords.words("english")
newStopWords = ['hotel','room','staff','rooms', 'stay', 'breakfast','location', "n't", "", "one", "us", 'desk', 'night', 'front', 'would', 'could', 'like', "'s"]
stops_hotel.extend(newStopWords)

In [29]:
#Counting words for rating 5.0
wordcount5 = {}
for word in list_filtered_5_new.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stops_hotel:
        if word not in wordcount5:
            wordcount5[word] = 1
        else:
            wordcount5[word] += 1

In [30]:
#Counting words for rating 1.0
wordcount1 = {}
for word in list_filtered_1_new.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stops_hotel:
        if word not in wordcount1:
            wordcount1[word] = 1
        else:
            wordcount1[word] += 1

In [31]:
#Counting words for rating 2.0
wordcount2 = {}
for word in list_filtered_2_new.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stops_hotel:
        if word not in wordcount2:
            wordcount2[word] = 1
        else:
            wordcount2[word] += 1

In [32]:
#Counting words for rating 3.0
wordcount3 = {}
for word in list_filtered_3_new.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stops_hotel:
        if word not in wordcount3:
            wordcount3[word] = 1
        else:
            wordcount3[word] += 1

In [33]:
#Counting words for rating 4.0
wordcount4 = {}
for word in list_filtered_4_new.lower().split():
    word = word.replace(".","")
    word = word.replace(",","")
    word = word.replace(":","")
    word = word.replace("\"","")
    word = word.replace("!","")
    word = word.replace("â€œ","")
    word = word.replace("â€˜","")
    word = word.replace("*","")
    if word not in stops_hotel:
        if word not in wordcount4:
            wordcount4[word] = 1
        else:
            wordcount4[word] += 1

In [34]:
wordcount = wordcount5, wordcount4, wordcount3, wordcount2, wordcount1

In [47]:
wordcount5

{'experience': 291,
 'rancho': 2,
 'valencia': 2,
 'absolutely': 84,
 'perfect': 317,
 'beginning': 15,
 'end': 67,
 'felt': 109,
 'special': 81,
 'happy': 170,
 'stayed': 600,
 'come': 154,
 'back': 421,
 'heart': 36,
 'beatamazing': 1,
 'place': 601,
 'everyone': 117,
 'extremely': 213,
 'warm': 87,
 'welcoming': 125,
 "'ve": 153,
 'top': 107,
 'notch': 32,
 'places': 113,
 'definitely': 398,
 '2': 225,
 'great': 1929,
 'romantic': 16,
 'getaway': 39,
 'take': 132,
 'kids': 121,
 'along': 53,
 'couple': 64,
 'stuffed': 4,
 'animals': 4,
 'waiting': 26,
 'girls': 20,
 'upon': 67,
 'arrival': 57,
 'ca': 99,
 'wait': 63,
 'go': 277,
 'backbooked': 1,
 '3': 149,
 'play': 18,
 'tennis': 10,
 'since': 94,
 'highest': 5,
 'rated': 9,
 'resorts': 7,
 'america': 7,
 'really': 394,
 'luxury': 33,
 'standpoint': 2,
 'overall': 116,
 'villas': 3,
 'attention': 24,
 'details': 10,
 'includes': 13,
 'fresh': 113,
 'squeezed': 1,
 'orange': 13,
 'juice': 13,
 'morning': 193,
 'restaurants': 316,
 '

In [51]:
for element, count in wordcount5:
    print(element, count)

ValueError: too many values to unpack (expected 2)

In [93]:
#n_print = int(input("How many most common words to print: "))
#print("\nOK. The {} most common words are as follows\n".format(n_print))
for element in wordcount:
    word_counter = collections.Counter(element) 
    print("----------------")
    for word, count in word_counter.most_common(5):
        print(word, ": ", count)

----------------
great :  1929
clean :  1530
nice :  1175
friendly :  1094
good :  992
----------------
good :  1222
clean :  1083
nice :  889
great :  881
friendly :  635
----------------
good :  588
clean :  397
nice :  335
great :  243
friendly :  215
----------------
good :  173
clean :  117
get :  112
bed :  111
nice :  108
----------------
dirty :  115
back :  114
bed :  109
never :  102
place :  101


In [96]:
word_counts = {"rating": [5.0, 5.0, 5.0, 5.0, 5.0 , 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0],
               "words": ["great","clean", "nice", "friendly", "good", "good", "clean", "nice", "great", "friendly", "good", "clean", "nice", "great", "friendly", "good", "clean", "get", "bed", "nice", "dirty", "back", "bed", "never", "place"], 
               "frequency": [1929, 1530, 1175, 1094, 992, 1222, 1083, 889, 881, 635, 588, 397, 335, 243, 215, 173, 117, 112, 111, 108, 115, 114, 109, 102, 101]}
 
most_common_df = pd.DataFrame(word_counts)
most_common_df

,rating,words,frequency
0,5.0,great,1929
1,5.0,clean,1530
2,5.0,nice,1175
3,5.0,friendly,1094
4,5.0,good,992
5,4.0,good,1222
6,4.0,clean,1083
7,4.0,nice,889
8,4.0,great,881
9,4.0,friendly,635


In [106]:
most_common_df_grouped = pd.DataFrame(most_common_df.groupby(["rating"]).min())
most_common_df_grouped

,words,frequency
rating,,
1.0,back,101
2.0,bed,108
3.0,clean,215
4.0,clean,635
5.0,clean,992


In [ ]:
# Create a dictionary
word_counts = {5: {“great” : 1929, “clean”: 1530}, 
               4: {“good”: 1222, “clean”: 1083}} 
# Convert the dictionary to a dataframe
df = pd.DataFrame(word_counts)

In [ ]:
# Create a data frame of the most common words 
# Draw a bar chart
lst = word_counter.most_common(n_print)
df_commonwords = pd.DataFrame(lst, columns = ['Word', 'Count'])
df_commonwords.plot.bar(x='Word',y='Count')